In [8]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean, median
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as tf
from tensorflow.keras import models
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU, LSTM, Conv1D, Conv2D
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

In [9]:
sp500 = {
    "index": "S&P500",
    "train": 62.54,
    "val": 68.53,
    "test": 60.07,
}

nasdaq = {
    "index": "NASDAQ",
    "train": 64.51,
    "val": 64.14,
    "test": 57.14,
}

In [31]:
def plot_accuracy():

    df = pd.DataFrame(columns=['stock', 'data', 'accuracy'])
    df = df.append({
        "stock": "S&P500",
        "data": "train",
        "accuracy": 62.54,
    }, ignore_index=True)
    df = df.append({
        "stock": "S&P500",
        "data": "val",
        "accuracy": 68.53,
    }, ignore_index=True)
    df = df.append({
        "stock": "S&P500",
        "data": "test",
        "accuracy": 60.07,
    }, ignore_index=True)
    
    df = df.append({
        "stock": "NASDAQ",
        "data": "train",
        "accuracy": 64.51,
    }, ignore_index=True)
    df = df.append({
        "stock": "NASDAQ",
        "data": "val",
        "accuracy": 64.14,
    }, ignore_index=True)
    df = df.append({
        "stock": "NASDAQ",
        "data": "test",
        "accuracy": 57.14,
    }, ignore_index=True)
    
    df = df.append({
        "stock": "US30",
        "data": "train",
        "accuracy": 60.24,
    }, ignore_index=True)
    df = df.append({
        "stock": "US30",
        "data": "val",
        "accuracy": 62.95,
    }, ignore_index=True)
    df = df.append({
        "stock": "US30",
        "data": "test",
        "accuracy": 54.76,
    }, ignore_index=True)
    
    fig = px.bar(df, x='stock', y='accuracy',
                 color='data', barmode='group',
                title="Binary Magnitude Prediction Accuracy for Each Index",
                text='accuracy')
    layout = {
       'xaxis': {'title': 'Stock Market Index'},
       'yaxis': {'title': 'Accuracy (%)', 'range': [0, 100]},
    }
    fig.add_hline(y=50, line_width=3, line_dash="dash", line_color="red")
    fig.update_layout(layout)
    fig.show()
plot_accuracy()

In [14]:
df = px.data.tips()
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2
